# A2A - a thread:

### What is A2A?
A2A is a protocol to allow AI agents to communicate autonomously amongst themselves in a standardized way. It was introduced by Google in April 2025 and donated to the Linux Foundation for more widespread adoption. As the sophistication of AI systems grows, so has the ability to automate daily tasks. However, scaling multi-agent systems has been a major bottleneck for many engineers. Frameworks like langgraph seek to solve this problem, but the framework is not very scalable and the api's don't always fit the needs of developers, can become outdated, and may introduce breaking changes...hence the need for custom agents, and modularity. A2A is an effort by Google to standardize inter-agent communication and allow developers to integrate the capabilities of various frameworks. 

The probabalistic origins of machine learning and AI models cause a significant amount of non-determinism in the outputs, on the contrary to traditional algorithms that provide a deterministic output (i.e. given the same input, it always produces the same output). Because of this, we can't always predict the output of a machine learning algorithm. So maybe it is obvious that in order to allow these systems to share information, the outputs must be wrapped in some metadata to provide some scaffolding for a message format, even though the messages themselves are always unique. 


See also MCP, a protocol introduced by Anthropic to provide a standard of communication between AI models and tools, data, prompts, and other resources. While MCP allows the AI "agent" external functionality like querying a database, searching the internet, or executing a workflow, A2A allows multiple specialized AI agents to communicate with each other and complete complex tasks by utilizing their variety of skills. If the A2A protocol reaches a critical mass, it will hopefully become the de-facto standard of communication for the new era of intelligent systems

Example: Google Search Agent, Database Query agent, summarizer agent, analysis agent.

### The Agent

So what is an Agent? While the A2A protocol seems like it has been standardized, the definition of "agent" certainly has not. Langchain and Microsoft define an [agent](https://langchain-ai.github.io/langgraph/agents/overview/) as "an LLM + tools + a prompt". Google has more broadly stated that an AI agent is a software system that uses AI (not just LLMs) to proactively pursue goals and complete tasks. This allows us to expand the scope of agents to include more specialized machine learning algorithms, although the implementation of Google's agent development kit ([ADK](https://google.github.io/adk-docs/)) indicates that they really fall into the first camp. For now, I will focus on the "LLM + tools + prompt" definition, but stay tuned for more exploration into nuanced algorithms in the future. 

Let's take a look at our first "agent"...

**Prerequisites**:
- Pyton 3.11+
- [Gemini API Key](https://aistudio.google.com/apikey)l

In [4]:
import os
from dotenv import load_dotenv

if not os.getenv('GOOGLE_GEMINI_API_KEY'):
    os.environ['GOOGLE_GEMINI_API_KEY'] = 'gemini-api-key'

load_dotenv()

True

In [5]:
from google.adk.agents import Agent
from google.adk.tools import google_search

my_first_agent = Agent(
    model="gemini-2.5-flash",
    name="My_first_agent",
    description="A simple agent that can call a google search",
    instruction="You are a helpful google search agent. Conduct a search when you determine it is necessary to do so.",
    tools=[google_search]
)

As you can see, there is not much effort to declare your first agent. Google's ADK abstracts away a lot of the headache. Each of the supplied parameters are pretty self-descriptive:
- Model: the large language model used as the intelligence engine for your agent
- Name: a descriptive name, can only contain letters, numbers and underscores
- Description: a helpful description can go a long way, especially once multiple agents are involved in a complex process
- Instruction: an optional system prompt that gets passed in as context to the LLM that contains any additional information or instructions
- Tools: a list of tools an agent is capable of using. In this case we have only outfitted our agent with the ability to conduct a google search

Cool, now we have a primitive agent. So how do we handle messages?

### Agent Executor
The Agent Executor interface is contained in the a2a library, and provides a wrapper for the Agent to handle requests. The interface contains two main methods: `execute()`, which handles the main execution logic of the Agent's runtime and `cancel()`, which can be called during the execution of a long-running task. For now, we will focus only on `execute()`, and leave `cancel()` to a future blog. Google provides a nice implementation for a generic ADK agent in their [A2A Samples](https://github.com/a2aproject) repo that I will borrow for this example.

```
class AgentExecutor(ABC):
    """Agent Executor interface.

    Implementations of this interface contain the core logic of the agent,
    executing tasks based on requests and publishing updates to an event queue.
    """

    @abstractmethod
    async def execute(
        self, context: RequestContext, event_queue: EventQueue
    ) -> None:
        """Execute the agent's logic for a given request context.

        The agent should read necessary information from the `context` and
        publish `Task` or `Message` events, or `TaskStatusUpdateEvent` /
        `TaskArtifactUpdateEvent` to the `event_queue`. This method should
        return once the agent's execution for this request is complete or
        yields control (e.g., enters an input-required state).

        Args:
            context: The request context containing the message, task ID, etc.
            event_queue: The queue to publish events to.
        """

    @abstractmethod
    async def cancel(
        self, context: RequestContext, event_queue: EventQueue
    ) -> None:
        """Request the agent to cancel an ongoing task.

        The agent should attempt to stop the task identified by the task_id
        in the context and publish a `TaskStatusUpdateEvent` with state
        `TaskState.canceled` to the `event_queue`.

        Args:
            context: The request context containing the task ID to cancel.
            event_queue: The queue to publish the cancellation status update to.
        """

```

The `execute()` method concurrently processes requests by keeping track of them with an `EventQueue`. Requests are presented in the form of a `RequestContext`, which contains the content of the request, along with other metadata. That is enough info for now, we will get into the weeds in future work.

A critical component to the successful operation of an Agent is the `session`. A session is a stateful container that allows agents to interact asynchronously. It manages the ongoing interaction between the agents and/or users, context such as memory and state, and a request/response loop to handle communication and execution of actions.



In [6]:
from a2a.server.agent_execution import AgentExecutor, RequestContext
from a2a.server.events import EventQueue
from a2a.server.tasks import TaskUpdater
from a2a.utils import new_task, new_agent_text_message
from a2a.types import TaskState, TextPart, Part

from google.adk.runners import Runner
from google.adk.artifacts import InMemoryArtifactService
from google.adk.sessions import InMemorySessionService
from google.adk.memory import InMemoryMemoryService
from google.genai import types

class MyAgentExecutor(AgentExecutor):

    def __init__(self, agent: Agent, status_message: str = "Executing task...", artifact_name: str = "response"):
        self.agent = agent
        self.status_message = status_message
        self.artifact_name = artifact_name
        self.runner = Runner(
            app_name=agent.name,
            agent=agent,
            artifact_service=InMemoryArtifactService(),
            session_service=InMemorySessionService(),
            memory_service=InMemoryMemoryService(),
        )


    async def execute(self, context: RequestContext, event_queue: EventQueue) -> None:
        
        query = context.get_user_input()
        task = context.current_task
        if not task:
            task = new_task(context.message)
        
        #enqueue a task to the event queue
        await event_queue.enqueue_event(task)

        updater = TaskUpdater(event_queue, task, task.contextId)

        try:
            await updater.update_status(
                state=TaskState.working,
                message=new_agent_text_message(
                    text="test test",
                    context_id=task.contextId,
                    task_id=task.id
                )
            )

            session = await self.runner.session_service.create_session(
                app_name=self.agent.name,
                user_id="a2a_user",
                state={},
                session_id=task.contextId,
            )

            content = types.Content(
                role='user',
                parts=[types.Part.from_text(query)]
            )

            response_text = ""
            async for event in self.runner.run_async(
                user_id="user",
                session_id=session.id,
                new_message=content
            ):
                if event.is_final_response() and event.content and event.content.parts:
                    for part in event.content.parts:
                        if hasattr(part, 'text') and part.text:
                            response_text += part.text + "\n"
                        elif hasattr(part, 'function_call'):
                            pass
            
            await updater.add_artifact(
                [Part(root=TextPart(text=response_text))],
                name=self.artifact_name
            )

            await updater.complete()
        
        except Exception as e:
            await updater.update_status(
                TaskState.failed,
                new_agent_text_message(f"Error: {e!s}", task.contextId, task.id),
                final=True
            )

    def cancel(self, context, event_queue):
        pass

Now that we can run an agent, how do we (and more importantly other agents, since this is a blog about A2A after all...) know what its capabilities are?

### Agent Card
The agent card is a public-facing JSON schema that exposes information and metadata to clients (users and other agents). It is like a user profile, but for AI agents. To define an agent card properly, you need:
- `name`
- `description`: description of the agent, its skills, and other useful information
- `version`
- `url`: the web endpoint where we can find our agent
- `capabilities`: supported A2A features like streaming or push notifications
- `skills`: A pillar of A2A, the skills discovered in the `AgentCard` come in a list of `AgentSkill` objects.

Lets give our agent an `AgentCard`:

In [7]:
from a2a.types import AgentCard, AgentCapabilities

agent_card = AgentCard(
    name=my_first_agent.name,
    description=my_first_agent.description,
    url="http://localhost:9999/",
    version='1.0',
    capabilities=AgentCapabilities(
        streaming=True
    ),
    defaultInputModes=["text", "text/plain"],
    defaultOutputModes=["text", "text/plain"],
    skills=[]
)

### Agent Skills
Agent skills describe specific capabilities the agent has, like searching the web, querying a database, executing an algorithmm...etc. Clients can find out what skills an agent has from the `AgentCard`. It's kind of like the agentic version of a resume. Skills have some attributes to define:
- `id`: a unique id
- `name`
- `description`: more detailed information about the skill's functionality
- `tags`: keywords
- `examples`: example usage of the skill
- `inputModes` and `outputModes`: supported modes for input and output, like text or json

Let's go back and define the Google search `AgentSkill` for our agent:

In [8]:
from a2a.types import AgentSkill

web_search_skill = AgentSkill(
    id='google_search',
    name='Google Search',
    description='Searches the web using the google_search tool',
    tags=['web search', 'google', 'search', 'look up']
)

agent_card.skills.append(web_search_skill)

Now our agent has advertized the ability to search the web on its agent card. We will take a deep dive into `AgentSkill`s, `AgentCapabilities`, and `AgentCard`s another time, but now lets zoom out a little bit. We have given lots of detail about our agent. Where it lives, what it can do, examples for how to use it... So how do we run it and start testing this stuff out?

### Starting the Server



In [ ]:
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.tasks import InMemoryTaskStore
from a2a.server.apps import A2AStarletteApplication

request_handler = DefaultRequestHandler(
    agent_executor=MyAgentExecutor(my_first_agent),
    task_store=InMemoryTaskStore()
)

server = A2AStarletteApplication(
    agent_card=agent_card,
    http_handler=request_handler,
    
)